# **Feed Forward Neural Network**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Standard libraries
import numpy as np
import pandas as pd
import time
import os

# For preprocessing
import tensorflow as tf

# For modeling
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, roc_auc_score, f1_score
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.utils import class_weight

# Operational
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import time
import random

In [ ]:
pkl_path = '/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/feature-extraction/Annotated/Regular/NotAveragePooled/split_features_3s_all_2D.pkl'
# pkl_path = 'C:/Users/thato/Documents/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/feature-extraction/Annotated/Regular/NotAveragePooled/split_features_3s_all_2D.pkl'

# Load the pickle file
with open(pkl_path, 'rb') as file:
    data = pickle.load(file)
del file

In [ ]:
path = '/content/drive/My Drive/Final-Year-Project/Final-Version-of-Bird-Classification-Project/a. Imbalanced-Data/3. Training/Figures/CNN/'

In [ ]:
train_data = data['train'].copy()
val_data = data['val'].copy()
del data

train_labels = train_data['label'].copy()
temp = train_data.copy()
del temp['label']
tr_features = temp

val_labels = val_data['label'].copy()
temp = val_data.copy()
del temp['label']
v_features = temp

## **Shuffling Data**

In [ ]:
def shuffle_data(input_label, input_features):
  input_len = len(input_label)
  np.random.seed(1826)
  input_indices = np.random.permutation(input_len)
  input_features = {key: np.array([input_features[key][i] for i in input_indices]) for key in input_features} # dictionary comprehension
  input_label = np.array([input_label[i] for i in input_indices])

  return input_label, input_features

In [ ]:
train_y, train_features = shuffle_data(train_labels, tr_features)

In [ ]:
val_y, val_features = shuffle_data(val_labels, v_features)

## **FFNN Model**

In [ ]:
def build_model(audio_features,
                hidden_layers,
                learning_rate,
                dropout_rate,
                regularizer_val,
                output_units=20
                ):

  tf.keras.backend.clear_session()

  # set input layer
  input = tf.keras.Input(shape=(audio_features.shape[1],), name='Input')

  # set hidden layers
  hidden_output = input

  for i, hidden_dim in enumerate(hidden_layers):
    hidden_output = tf.keras.layers.Dense(units=hidden_dim,
                                          activation='relu',
                                          kernel_regularizer=tf.keras.regularizers.l2(regularizer_val),
                                          name=f'hidden{i}')(hidden_output)
    hidden_output = tf.keras.layers.Dropout(rate=dropout_rate, name=f'dropout{i}')(hidden_output)

  # set output layer
  output = tf.keras.layers.Dense(units=output_units,
                                 activation='softmax',
                                 name='Output')(hidden_output)
  # create model
  model = tf.keras.models.Model(inputs=input, outputs=[output])

  # compile model
  model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                metrics=['accuracy'])

  return model

In [ ]:
def visualize(model_history, name):
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))

    # Visualize the loss
    axes[0].plot(model_history.history['loss'], color='red', label='Training Loss')
    axes[0].plot(model_history.history['val_loss'], color='blue', label='Validation Loss')
    axes[0].set_xlabel('Epoch', fontsize=12)
    axes[0].set_ylabel('Loss', fontsize=12)
    axes[0].set_title('Loss Progression', fontsize=14)
    axes[0].grid(True)
    axes[0].legend()

    # Visualize the accuracy
    axes[1].plot(model_history.history['accuracy'], color='green', label='Training Accuracy')
    axes[1].plot(model_history.history['val_accuracy'], color='orange', label='Validation Accuracy')
    axes[1].set_xlabel('Epoch', fontsize=12)
    axes[1].set_ylabel('Accuracy', fontsize=12)
    axes[1].set_title('Accuracy Progression', fontsize=14)
    axes[1].grid(True)
    axes[1].legend()

    plt.savefig(f'{path+name}_3s_model_training_history_2D_annotated.pdf')

    # Fine-tune layout and display the plots
    plt.tight_layout()
    plt.show()

In [ ]:
def evaluate_model(val_y, val_yhat, val_yhat_result, num_classes=20):

    print('Validation classification Report \n')
    print(classification_report(val_y, val_yhat_result))

    # Calculate AUC for multiclass classification using 'ovr' and 'weighted' average
    auc_score = roc_auc_score(val_y, val_yhat, multi_class='ovr', average='weighted')
    print(f'AUC Score: {auc_score}')

    # Calculate F1-score with 'weighted' average for imbalanced dataset
    f1 = f1_score(val_y, val_yhat_result, average='weighted')
    print(f'F1 Score (Weighted): {f1}')

    val_score = {'f1': f1, 'auc': auc_score}

    return val_score

In [ ]:
def tile_and_crop(feature, target_size):
    tiled = np.tile(feature, (1, target_size // feature.shape[1] + 1, 1))
    return tiled[:, :target_size, :]

In [ ]:
def build_and_train_models(model_configs, train_features, train_y, val_features, val_y, train_results, val_results, val_scores, epochs=100, batch_size=32):
    """
    Build, train, and evaluate multiple models based on configurations provided in model_configs.

    Parameters:
    - model_configs: Dictionary where each key is a model name and the value is a config dictionary for that model.
    - train_features: Training features (e.g., mel spectrograms + MFCC).
    - train_y: Training labels.
    - val_features: Validation features (e.g., mel spectrograms + MFCC).
    - val_y: Validation labels.
    - train_results: Dictionary to store training results (e.g., accuracy).
    - val_results: Dictionary to store validation results (e.g., accuracy).
    - val_scores: Dictionary to store validation metrics like precision, recall, etc.
    - epochs: Number of epochs to train each model.
    - batch_size: Batch size for training.

    Returns:
    - None (modifies train_results, val_results, and val_scores in place).
    """

    for model_name, config in model_configs.items():
        print(f"Building and training {model_name}...")

        # Unpack the configuration for each model
        learning_rate = config.get('learning_rate')
        dropout_rate = config.get('dropout_rate')
        hidden_layers = config.get('hidden_layers')
        regularizer_val = config.get('regularizer_val')

        # Build the model
        model = build_model(
            audio_features=train_features,
            hidden_layers=hidden_layers,
            learning_rate=learning_rate,
            dropout_rate=dropout_rate,
            regularizer_val=regularizer_val
        )

        # Train the model
        history = model.fit(
            x=train_features,
            y=train_y,
            epochs=epochs,
            validation_data=(val_features, val_y),
            batch_size=batch_size,
            verbose=1
        )

        # Visualize training history
        visualize(history, model_name)

        # Evaluate model on training data
        train_yhat = model.predict(train_features)
        train_yhat_result = np.argmax(train_yhat, axis=-1)
        train_results[model_name] = model.evaluate(train_features, train_y)[-1]

        # Evaluate model on validation data
        val_yhat = model.predict(val_features)
        val_yhat_result = np.argmax(val_yhat, axis=-1)
        val_results[model_name] = model.evaluate(val_features, val_y)[-1]

        # Store validation scores (like accuracy, precision, recall, etc.)
        val_scores[model_name] = evaluate_model(val_y=val_y, val_yhat=val_yhat, val_yhat_result=val_yhat_result)

        print(f"Completed training and evaluation for {model_name}.\n")

In [ ]:
model_configs = {
    'model_1': {
        'hidden_layers': [128, 64, 32],
        'learning_rate': 0.0001,
        'dropout_rate': 0.3,
        'regularizer_val': 0.15
    },
    'model_2': {
        'hidden_layers': [256, 128, 64],
        'learning_rate': 0.0002,
        'dropout_rate': 0.4,
        'regularizer_val': 0.2
    },
    'model_3': {
        'hidden_layers': [256, 64, 32],
        'learning_rate': 0.00005,
        'dropout_rate': 0.5,
        'regularizer_val': 0.05
    },
    'model_4': {
        'hidden_layers': [128, 128, 64],
        'learning_rate': 0.0003,
        'dropout_rate': 0.5,
        'regularizer_val': 0.1
    },
    'model_5': {
        'hidden_layers': [64, 64, 32],
        'learning_rate': 0.0001,
        'dropout_rate': 0.35,
        'regularizer_val': 0.15
    },
    'model_6': {
        'hidden_layers': [256, 128],
        'learning_rate': 0.0001,
        'dropout_rate': 0.4,
        'regularizer_val': 0.12
    }
}


In [ ]:
train_results = {}
val_results = {}

val_scores = {}

### With Mel Spectrogram, MFCCs, Chroma

In [ ]:
training_features_3D = np.concatenate((train_features['mfcc'], train_features['chroma'], train_features['melspectrogram']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(5278, 63196)

In [ ]:
validation_features_3D = np.concatenate((val_features['mfcc'], val_features['chroma'], val_features['melspectrogram']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(1350, 63196)

In [ ]:
# Build, train, and evaluate models
build_and_train_models(
    model_configs=model_configs,
    train_features=training_features,
    train_y=train_y,
    val_features=validation_features,
    val_y=val_y,
    train_results=train_results,
    val_results=val_results,
    val_scores=val_scores,
    epochs=100,
    batch_size=32
)

Checking the class weights impact on the best model

In [ ]:
class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                  classes=np.unique(train_y),
                                                  y=train_y)
class_weight_dict = dict(enumerate(class_weights))

In [ ]:
history = model.fit(training_features, train_y,
                    validation_data=(validation_features, val_y),
                    epochs=100,
                    batch_size=32,
                    class_weight=class_weight_dict,
                    verbose=1)

## Review all the results

In [ ]:
train_results_df = pd.DataFrame(list(train_results.items()), columns=['Features', 'Train_Accuracy']).round(3)
val_results_df = pd.DataFrame(list(val_results.items()), columns=['Features', 'Val_Accuracy']).round(3)

result_df = train_results_df.merge(val_results_df, on='Features')
result_df = result_df.sort_values('Features')
result_df

,Features,Train_Accuracy,Val_Accuracy
0,model_1,1.000,0.681
1,model_2,0.996,0.656
2,model_3,1.000,0.707
3,model_4,0.998,0.716
4,model_5,0.986,0.685


In [ ]:
val_scores_df = pd.DataFrame([(key, value['f1'], value['auc']) for key, value in val_scores.items()],
                             columns=['Features', 'F1_Score', 'AUC_Score']).round(3)

val_scores_df = val_scores_df.sort_values('Features')
print(val_scores_df)

  Features  F1_Score  AUC_Score
0  model_1     0.682      0.956
1  model_2     0.664      0.961
2  model_3     0.704      0.971
3  model_4     0.714      0.970
4  model_5     0.683      0.960
